# Descartes Underwriting technical test

Please, see the notebook "Descartes Underwriting test - Model generation" first.

## I) Librairies, folder, function

In [1]:
import pandas as pd
import numpy as np
from pickle import load

##### <font color='red'>ENTER YOUR FOLDER HERE</font>

In [2]:
path = r'C:\Users\erwan_000\Descartes Underwriting\auto-insurance-fall-2017'

In [3]:
def one_hot_enconding(df,column_name_to_ohe):
    if column_name_to_ohe in df.columns:
        one_hot = pd.get_dummies(df[column_name_to_ohe])
        df = df.drop(column_name_to_ohe,axis = 1)
        df = df.join(one_hot)
    else:
        print("/!\ Column %s doesn't exist in the dataframe. No modification" %column_name_to_ohe)
    return df

def configure_df_to_predict(df):
    # same modifications as in the notebook "Descartes Underwriting test - Model generation"
    df['INCOME']=df['INCOME'].replace('[\$,]', '', regex=True).astype(float)
    df['PARENT1']=df['PARENT1'].replace({'No':0,'Yes':1})
    df['HOME_VAL']=df['HOME_VAL'].replace('[\$,]', '', regex=True).astype(float)
    df['MSTATUS']=df['MSTATUS'].replace({'z_No':0,'Yes':1})
    df['SEX']=df['SEX'].replace({'z_F':0,'M':1})
    df=one_hot_enconding(df,'EDUCATION')
    df=one_hot_enconding(df,'JOB')
    df['CAR_USE']=df['CAR_USE'].replace({'Private':0,'Commercial':1})
    df['BLUEBOOK']=df['BLUEBOOK'].replace('[\$,]', '', regex=True).astype(float)
    df=one_hot_enconding(df,'CAR_TYPE')
    df['RED_CAR']=df['RED_CAR'].replace({'no':0,'yes':1})
    df['OLDCLAIM']=df['OLDCLAIM'].replace('[\$,]', '', regex=True).astype(float)
    df['REVOKED']=df['REVOKED'].replace({'No':0,'Yes':1})
    df.loc[df['CAR_AGE'] < 0, 'CAR_AGE']=np.nan
    df['URBANICITY']=df['URBANICITY'].replace({'Highly Urban/ Urban':0,'z_Highly Rural/ Rural':1})
    del df['TARGET_AMT']
    del df['INDEX']
    df = df.fillna(df.mean())
    df = df[['z_High School', 'Manager', 'z_Blue Collar', 'CAR_AGE', 'KIDSDRIV', 'AGE', 'HOMEKIDS', 'INCOME',
             'PARENT1', 'HOME_VAL', 'MSTATUS', 'CAR_USE', 'BLUEBOOK', 'OLDCLAIM', 'CLM_FREQ', 'REVOKED', 'MVR_PTS',
             'URBANICITY']]
    return df    

## II) Data import and configuation

In [4]:
df = pd.read_csv(path + '\\test_auto.csv')

In [5]:
df_to_predict = configure_df_to_predict(df.loc[:, df.columns != 'TARGET_FLAG'].copy())

In [6]:
# scaler used for model creation
scaler = load(open(path + '\descartes_underwriting_scaler.pkl', 'rb'))
# applying the scaler to df_to_predict
df_to_predict = scaler.transform(df_to_predict)

## III) Predictions

In [7]:
model = load(open(path + '\descartes_underwriting_model.pkl', 'rb'))

In [8]:
predictions = list(model.predict(df_to_predict))

In [9]:
df_results = pd.DataFrame(df['INDEX'].copy())
df_results['prediction'] = predictions
df_results.to_csv(path  + '\df_results.csv', index = False)